## Workspace setup

In [1]:
from datetime import datetime 
import uproot
import awkward as ak
import tensorflow as tf
import numpy as np
import importlib
from functools import partial
from os.path import isfile
import io_functions as io


from tensorflow.data import Dataset, TFRecordDataset
from tensorflow.io import TFRecordWriter, TFRecordOptions
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

2023-11-14 22:47:55.381630: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## TFRecord creation

In [2]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [5]:
%%time

importlib.reload(io)

dataPath = '/scratch/szslaw/'
train_files = [dataPath+'out_random_sigma-001.root:TPCData']
batchSize = 1

datasetGenerator = io.generator(files=train_files, batchSize=batchSize)

CPU times: user 2.19 ms, sys: 156 µs, total: 2.35 ms
Wall time: 1.48 ms


In [6]:
def serialize(charge_array, target):
  feature = {'myChargeArray' : _bytes_feature(charge_array.tobytes()),
             'target' : _bytes_feature(target.tobytes())}
  example = tf.train.Example(features=tf.train.Features(feature=feature))
  return example.SerializeToString()

In [7]:
%%time

outputFile = 'out_random_sigma-001.tfrecord'

if isfile(dataPath + outputFile):
    raise Exception('outputFile already exists')

options = TFRecordOptions(compression_type='GZIP')

writer = TFRecordWriter(dataPath + outputFile, options=options)
for charge_array, target in datasetGenerator:
  example = serialize(charge_array, target)
  writer.write(example)

2023-11-14 19:42:48.972928: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: UNKNOWN ERROR (34)


CPU times: user 2h 9min 24s, sys: 1min 10s, total: 2h 10min 35s
Wall time: 2h 10min 53s


### Read TFRecord

In [6]:
raw_dataset = TFRecordDataset(dataPath + 'out_random_sigma-001.tfrecord', compression_type='GZIP')

# Create a description of the features.
feature_description = {
    'myChargeArray': tf.io.FixedLenFeature([], tf.string),
    'target': tf.io.FixedLenFeature([], tf.string),

}

def _parse_function(example_proto, batchSize):
  # Parse the input `tf.train.Example` proto using the dictionary above.
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    charge, target = parsed_features['myChargeArray'], parsed_features['target']
    # decode from bytes
    charge = tf.io.decode_raw(charge, 'float64')
    target = tf.io.decode_raw(target, 'float64')
    # reshape
    charge = charge.reshape([batchSize, 256, 512, 3])
    target = target.reshape([batchSize, 9])
    
    return charge, target

parser = partial(_parse_function, batchSize=batchSize)

parsed_dataset = raw_dataset.map(parser)
